## Domácí úkol - Batoh

Za domácí úkol budete mít vyřešit pomocí evolučního algoritmu problém batohu. Ten spočívá v tom, že máme batoh kapacity K a N předmětů, každý s cenou c<sub>i</sub> a objemem v<sub>i</sub> a chceme vybrat takové věci, abychom maximalizovali zisk a zároveň abychom nepřekročili kapacitu batohu. 

Vstupní data máte ve složce *domaci_ukol_data*. Obsahuje čtyři soubory s daty a dva s výsledky. Na první řádce souboru s daty je vždy počet předmětů a kapacita batohu oddělené mezerou, každý další následující řádek obsahuje cenu a objem předmětu taktéž oddělené mezerou. První dva soubory slouží pro snažší odladění evolučního algoritmu a obsahují i k sobě extra soubory s optimálním řešením. Na dalších dvou máte za úkol algoritmus pustit a výsledky na nich naměřené mi poslat. 

Napište tedy nějaký svůj evoluční algoritmus, který bude řešit problém batohu a pusťte ho na vstupních datech. Svůj kód, popis evolučního algoritmu (zvolené evoluční operátory, kódování jedince, atd.) a rozbor výsledků, včetně nejlepšího dosaženého skóre i s jejich odůvodněním mi pošlete emailem do stanoveného deadline.  Pro sepsání popisu vašeho evolučního algoritmu, parametrů evoluce, zvolené reprezentace jedince a rozboru výsledků použijte [tento template](https://github.com/kackamac/Prirodou-inspirovane-algoritmy/blob/master/04_spojita_reprezentace/DU1_evolucni_algoritmy.pdf).

##### Importy

In [2]:
import array
import random
import numpy as np
import math

from deap import algorithms
from deap import base
from deap import creator
from deap import tools

import my_algorithms

Inicializácia random itemov a veľkosť batohu

In [3]:
# random.seed(64)
random.seed()
items_count = 20

weight_bounds = (10,200)
price_bounds = (10,200)

weights = random.sample(range(weight_bounds[0],weight_bounds[1]), items_count)
prices = random.sample(range(price_bounds[0],price_bounds[1]), items_count)

max_weight = 1000

Načítanie batohu zo súboru

In [4]:
#import weights and prices from txt file

def load_input_data(file_path):

    weights = []
    prices = []

    with open(file_path) as file:
        line_split = file.readline().split(" ")
        items_count = int(line_split[0])
        max_weight = int(line_split[1])
        for i in range(items_count):
            line_split = file.readline().split(" ")
            prices.append(int(line_split[0]))
            weights.append(int(line_split[1]))

    return weights, prices, max_weight

### Sedond try - permutation individual

max fitness - berieme itemy v poradí permutácie, kým nenaplníme batoh. Ostatné neberieme

In [5]:
def fitness_batoh_perm(individual):
    total_price = 0
    total_weight = 0
    for i in individual:
        total_weight += weights[i]
        if total_weight > max_weight:
            return total_price,
        total_price += prices[i]
    return total_price,

In [6]:
def fitness_batoh_perm_2(individual):
    total_price = 0
    total_weight = 0
    for i in individual:
        total_weight += weights[i]
        if total_weight <= max_weight:
            total_price += prices[i]
        else:
            total_weight -= weights[i]
    return total_price,

In [16]:
def mut_transposition(individual, indv_size, transposition_count):
    count = random.randint(1, transposition_count)

    for i in range(count):
        pos_1 = random.randint(0, indv_size - 1)
        pos_2 = random.randint(0, indv_size - 1)

        individual[pos_1], individual[pos_2] = individual[pos_2], individual[pos_1]

    return individual,

In [17]:
weights, prices, max_weight = load_input_data(file_path = "./domaci_ukol_data/input_data_1000.txt")
indv_size = len(weights)

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

toolbox.register("indices", random.sample, range(indv_size), indv_size)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)

toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", fitness_batoh_perm)

toolbox.register("mate", tools.cxPartialyMatched)
toolbox.register("mutate", mut_transposition, indv_size=indv_size)
# toolbox.register("select", tools.selRoulette)
toolbox.register("select", tools.selTournament, tournsize = 3)

C:\Users\llama\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
C:\Users\llama\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [18]:
POP_SIZE = 500
GEN_COUNT = 5000


pop = toolbox.population(n=POP_SIZE)
hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.8, mutpb=0.3, ngen=GEN_COUNT,
                               stats=stats, halloffame=hof, verbose=True)

print(hof)
print(fitness_batoh_perm(hof[0]))

gen	nevals	avg    	std 	min 	max 
0  	500   	4618.88	1185	1543	9093
1  	421   	5335.16	1169.37	2180	9963
2  	426   	5933.88	1105.95	3010	9963
3  	443   	6176.24	1278.24	2500	10354
4  	429   	6491.39	1336.79	2088	9963 
5  	431   	6770.44	1440.26	1631	9963 
6  	424   	6960.37	1528.26	1730	10236
7  	430   	7036.26	1643.76	1980	11241
8  	424   	7315.13	1686.91	2139	11237
9  	422   	7510.43	1629.79	2070	11237
10 	439   	7701.26	1675.33	2333	11237
11 	430   	7924.58	1623.16	2453	11612
12 	440   	8166.49	1697.89	2607	11441
13 	447   	8138.63	1948.07	2624	12220
14 	423   	8510.94	1830.09	2869	13219
15 	411   	8792.83	1778.32	3282	12740
16 	436   	8770.61	2000.66	3017	12740
17 	441   	8864.91	1917.08	3414	13539
18 	430   	9022.97	1834.48	2531	13539
19 	401   	9212.98	1923.1 	3469	13539
20 	431   	9161.93	1934.66	2897	13539
21 	426   	9396.81	1960.43	2277	13539
22 	432   	9476.6 	1944.1 	3458	13539
23 	451   	9679.92	1948.31	2920	13539
24 	442   	10067  	1862.61	3469	13939
25 	429   	10583.6	172

In [9]:
print(hof)
print(fitness_batoh_perm(hof[0]))

[[419, 60, 494, 281, 585, 523, 480, 10, 739, 821, 822, 273, 886, 215, 703, 824, 469, 37, 989, 836, 13, 736, 216, 830, 446, 30, 732, 362, 121, 38, 708, 476, 379, 751, 845, 657, 32, 610, 914, 770, 855, 146, 967, 987, 493, 945, 347, 603, 23, 599, 249, 48, 373, 25, 737, 53, 426, 382, 937, 134, 743, 473, 573, 612, 421, 786, 992, 254, 775, 334, 6, 463, 269, 986, 245, 669, 903, 218, 462, 594, 307, 876, 704, 561, 214, 923, 791, 391, 435, 782, 74, 294, 157, 749, 67, 486, 919, 290, 746, 910, 108, 816, 692, 93, 355, 950, 965, 747, 182, 705, 864, 141, 819, 779, 130, 530, 781, 336, 319, 752, 321, 626, 902, 856, 981, 767, 104, 983, 410, 633, 638, 607, 514, 284, 938, 209, 582, 978, 96, 590, 41, 837, 899, 76, 729, 250, 208, 478, 467, 892, 867, 370, 289, 318, 616, 900, 206, 470, 809, 839, 62, 551, 940, 366, 663, 155, 636, 201, 921, 453, 305, 360, 398, 50, 712, 865, 128, 961, 465, 872, 456, 268, 563, 776, 39, 524, 861, 440, 219, 98, 642, 798, 578, 139, 879, 310, 728, 149, 941, 438, 186, 721, 780, 272, 2

#### first try - binárny jedinci

max fitness (cena ju zvyšuje a váha prekročená cez max nosnosť batohu prináša postihy)

In [ ]:
def fitness_batoh_bin(individual):
    total_price = np.sum(np.array(individual) @ np.array(prices))
    total_weight = np.sum(np.array(individual) @ np.array(weights))

    if total_weight <= max_weight:
        return total_price,
    else:
        return 1,

Vytvorenie prostredia (knižnica Deap)

In [ ]:
weights, prices = load_input_data(file_path = "./domaci_ukol_data/input_data_100.txt")
indv_size = len(weights)

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", array.array, typecode='b', fitness=creator.FitnessMax)

toolbox = base.Toolbox()

toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, indv_size)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", fitness_batoh_bin)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.2)
toolbox.register("select", tools.selRoulette)

Final part

In [ ]:
pop = toolbox.population(n=500)
hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=200, 
                               stats=stats, halloffame=hof, verbose=True)